In [ ]:
import shutil
from os import path, getcwd, chdir, makedirs, listdir 
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('train.csv')

class_names = df.species.unique()
num_classes = len(class_names)

# setup folders for each class

for class_name in class_names:
    path_ = path.join('images',class_name)
    if not path.exists(path_):
        makedirs(path_)
    
    
# copies image files to species specific subfolders

for i in range(len(df)):
    Class = df.iloc[i,1]
    File = df.iloc[i,0]
    path_ = f'images/{Class}/{File}'
    if not path.exists(path_):
        shutil.copy(f'train_images/{File}', f'images/{Class}') 

In [ ]:
df = pd.read_csv('train.csv')

# setup folders per class and subfolders for individuals within class  and move image files into those folders 

for i in range(len(df)):
    Class = df.iloc[i,1]
    Identity = df.iloc[i,2]
    File= df.iloc[i,0]
    filepath = f'test/{Class}/{Identity}'
    file=path.join(filepath,File)
    makedirs(filepath, exist_ok=True)
    if not path.exists(file):
        shutil.copy(f'train_images/{File}',filepath)

In [ ]:
img_height = 128
img_width = 128

num_classes = 30

model_all = models.Sequential([
    
    layers.Conv2D(filters=32, kernel_size=(3,3), activation= 'relu', input_shape=(img_height,img_width,3)),
    layers.MaxPooling2D(2,2,),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation= 'relu', input_shape=(img_height,img_width,3)),
    layers.MaxPooling2D(2,2,),


       

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes, activation='softmax')

])
#model_all.summary()
opt = optimizers.Adam(.001)
model_all.compile(optimizer = opt,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


checkpoint_filepath = './checkpoints/checkpoint_all.ckpt'
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    verbose = 1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
batch_size = 10


train_datagen = ImageDataGenerator(rescale=1/255,
    rotation_range=20,
    zoom_range=.2,
    shear_range=20,
    horizontal_flip=True,
    fill_mode='nearest',                  
    validation_split=0.2)


train_generator = train_datagen.flow_from_directory(
    'images',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    'images', # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

In [ ]:
history = model_all.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 25,
    callbacks=[model_checkpoint_callback])

history.epoch, history.history['accuracy'][-1]

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
    'train_images', #will need to change to test_images
    target_size=(img_height, img_width),
    class_mode=None,
    shuffle='False'
   
) 


yAll_pred=model.predict(test_generator, verbose=1, steps=2) 
yAll_labels = np.argmax(y_pred,axis=1)
labelsAll = class_names[yAll_labels]
filesAll=test_generator.filenames
#print(y_labels)
#print(labels[0:10])
#print(files[0:10])


# The model weights (that are considered the best) are loaded into the model.
#model.load_weights(checkpoint_filepath)

In [ ]:
img_height = 256
img_width = 256


for classes in class_names:

              
    checkpoint_filepath = f'./checkpoints/checkpoint_{classes}.ckpt'
    model_checkpoint_callback = callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        verbose = 1,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    num_classes = len(listdir(f'test/{classes}'))
    print(num_classes)
    print(classes)
    
    model = models.Sequential([
    
        layers.Conv2D(filters=32, kernel_size=(3,3), activation= 'relu', input_shape=(img_height,img_width,3)),
        layers.MaxPooling2D(2,2,),
        layers.Conv2D(filters=64, kernel_size=(3,3), activation= 'relu', input_shape=(img_height,img_width,3)),
        layers.MaxPooling2D(2,2,),
        layers.Conv2D(filters=128, kernel_size=(3,3), activation= 'relu', input_shape=(img_height,img_width,3)),
        layers.MaxPooling2D(2,2,),
        
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),

    ])
    
    #model.summary()
    opt = optimizers.Adam(.00001)
    model.compile(optimizer = opt,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


    batch_size = 10


    train_datagen = ImageDataGenerator(rescale=1/255,
        rotation_range=20,
        zoom_range=.2,
        shear_range=20,
        horizontal_flip=True,
        fill_mode='nearest',                  
        validation_split=0.99)


    train_generator = train_datagen.flow_from_directory(
        f'test/{classes}',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training') 

    validation_generator = train_datagen.flow_from_directory(
        f'test/{classes}', # same directory as training data
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation') 

    history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs = 15,
        callbacks=[model_checkpoint_callback])

    history.epoch, history.history['accuracy'][-1]